# ∆F Strings for Formatting

## Description


<div class="alert alert-block alert-info">
    <font color="purple"><b>∆F</b></font> is a <font color="red"><b><i>prototype</i></b></font> for a convenient formatting utility for Dyalog APL.  Expanding on ideas from other languages, such as Python's <i>f-strings</i>, <font color="purple"><b>∆F</b></font> displays multiline text, objects of various types, ranks and depth,arbitrary code, and integrated custom formatting, with an <font color="dark-red"><b><i>APL</i></b></font> flair.<br>
    <font color="purple"><b>∆F</b></font> is designed to be useful for assertions (which display values only when boolean conditions are met), debugging, and routine display, expanding on existing capabilities of <b>⎕FMT</b> and other APL tools.
</div>

# Table of Contents
* [Getting Started](#Getting-Started)
* [∆F Fields: Text, Code, and Space Fields](#∆F-Fields) 
  * [Text Fields](#Text-Fields)
      * Newlines and Escape Sequences: `\⋄` and `\{`  `\}`  `\`
  * [Code Fields: Basics](#Code-Fields:-Basics)
      * Simple Variables: `{var1} {var2}`
      * Debug Mode:  `'debug' ∆FMT ...`
      * Code Fields Are DFNS: `{ c+x×b+x×a }`
        * Performing arbitrary calculations: `{¯1+2*31}`
      * Comments in Code Fields `{n*0.5 ⍝ sqrt(n)}`
      * [Strings in Code Fields Require Double Quotes](#DQ-Strings-Required-in-Code-Fields)
        * `{↑"line1" "line2"}`
        * Escapes in DQ Strings: `{"line1\⋄line2"}`
  * [Space Fields](#Space-Fields)
      * Basic Space Fields: `{   }` and `{}`
      * Comments in Space Fields: `{   ⍝ 3 spaces }`
      * Examples
      * Why Space Fields
      * Explicit Width Counts `{ :30: }` and `{ :30: ⍝ 30 spaces }`
  * [Code Fields: Advanced Topics](#Code-Fields:-Advanced-Topics)
      * [Pseudobuiltin &#36; for Detailed formatting](#Pseudobuiltin-\$-for-Detailed-Formatting)
      * [Pseudobuiltin &#36;&#36; for a Boxed Display](#Pseudobuiltin-\$\$-for-a-Boxed-Display) 
      * [Shortcuts for Accessing Elements of ∆F Right-Argument ⍵](#Shortcuts-for-Accessing-Elements-of-⍵)
        * ⍹0, ⍹1, ..., and ⍹ (and aliases ⍵0, ⍵1, ..., and ⍵_).
      * [Justification and Centering with &#36;](#Justification-and-Centering-with-\$)
        * Specifiers `L`, `C`, `R`; `l`, `c`, `r`
      * [Self-Documenting Code Fields](#Self-Documenting-Code-Fields) 
        * `{code → }`
        * With comments: `{code ... ⍝ comment ⋄ → }` 
      * [Pseudobuiltin &#36;&#36;&#36; for Adding Quotes or Decorators](#Pseudobuiltin-$$$-for-Adding-Quotes-or-Decorators)
      * [Pseudobuiltin % for Formatting Dates and Times](#Pseudobuiltin-%-for-Formatting-Dates-and-Times)
      * [Code Fields: Private Namespace ⍺](#Code-Fields:-Private-Namespace-⍺)
* [∆F and Looping](#∆F-and-Looping)
* [Assertions with ∆F](#Assertions-with-∆F)
* [∆F Syntax Summary](#∆F-Syntax-Summary)
***

## Getting Started
Let's get started!  



Let's be sure the file __∆Format.dyalog__ is accessible and loaded. When fixed, it creates a single function `∆F`. And let's be sure the environment is set up. ***We'll assume an Index Origin of 0 (`⎕IO←0`) for this session.***

In [91]:
⍝ ⎕←  '0. Our active directory is ',⎕SH 'pwd'
 '1. Does ∆Format.dyalog exist in this directory? ', 'No' 'Yes'⊃⍨ ⎕NEXISTS '∆Format.dyalog' 
 '2. Loading ∆Format.dyalog as:',(2 ⎕FIX 'file://∆Format.dyalog')
 '3. ','∆F utility now exists!' 'Whoops! No ∆F function.' ⊃⍨3≠⎕NC '∆F'

⍝ Set the usual system vars to something reasonable...
  ⎕IO← 0 ⋄ ⎕ML← 1 ⋄ ⎕PP← 6 ⋄ ⎕FR← 645

1. Does ∆Format.dyalog exist in this directory? Yes

2. Loading ∆Format.dyalog as: ∆Format

3. ∆F utility now exists!

Before showing how ∆F works, let's create a couple of variables...<BR>
Here are the vectors `string` and `numbers`:

In [92]:
string  ← 'This is a string'
numbers ← 10 20

We will display them using ∆F.

In [93]:
∆F 'My string = "{string}".  My numbers = {numbers}.'

My string = "This is a string". My numbers = 10 20.

We can do this more concisely using **Self-documenting Code Fields** (thanks to Python), signaled by the trailing right arrow *before* the closing curly brace: `{code → }`. Spaces adjacent to the arrow are optional, but may make the formatted output more readable. **Self-documenting Code Fields** are discussed further below. 
_(Note: The symbol `➤` is a special right arrow that separates the literal code from its value.)_

In [94]:
∆F '{string → }.  {numbers → }'

string ➤ This is a string. numbers ➤ 10 20

But, let's start at the __beginning__!

## ∆F Fields
Text Fields, Code Fields, and Space Fields
### Text Fields
`'A simple string'`

The simplest possible format string-- we'll call it an __∆F__ string-- consists of a simple __Text__ field.

In [95]:
∆F 'This is a simple string.'

This is a simple string.

#### Text Fields are 2D
Like `⎕FMT`-formatted objects, text fields are always 2-dimensional (matrices).

In [96]:
⍴∆F 'This is a simple string.'

1 24

#### Text Fields are 2D: Newlines and Escapes
__Text__ fields can consist of one or more lines, each separated by the special newline escape sequence `\⋄`. Using __Text__ fields this way is just one way to create a 2D list of items or a multiline paragraph. ∆F always returns a character matrix of 0 or more rows.

In [97]:
∆F 'This\⋄is a\⋄multiline string.'

This 
is a 
multiline string.

You can insert most any Unicode character into a __Text__ field. Only four characters (with special meaning _described below_) require special treatment: 

`⋄`, `{`, `}`, _and_ `\`.  
   
A diamond `⋄` _not_ preceded by a backslash `\` has no special meaning; only the escaped sequence `\⋄` denotes a newline.  You'll see below that a bare `{` begins a __Code__ field, which is terminated by a balancing bare `}`. So, if you want to enter a left brace literal, escape it with \\, as here.
 


In [98]:
∆F 'This is a diamond ⋄, an opening brace \{, a closing brace \}, and a backslash \\.' 

This is a diamond ⋄, an opening brace {, a closing brace }, and a backslash \.

Finally, literal backslashes only need to be escaped themselves (i.e. as `\\`) when right before a `⋄`, `{`, `}`, or another backslash `\`. In other cases, such as `+\⍳` no extra backslash is required.

In [99]:
∆F '"We can sum the numbers ⍵ via +\⍵"'
∆F '"We can sum the numbers ⍵ via +\\⍵"'

"We can sum the numbers ⍵ via +\⍵"

"We can sum the numbers ⍵ via +\⍵"

### Code Fields: Basics 
#### Code Fields: Simple Variables `'{lastName}, {firstName}'`
Let's create a more useful example of ∆F strings using the following three variables. 

In [100]:
what←  'This'
type←  'simple'
thing← 'string'

Within separate sets of curly braces `{..}`, which delimit a __Code__ field, we include the three variable names: `what`, `type`, and `thing`. We'll say more about __Code__ fields in a moment.

In [101]:
∆F '{what} is a {type} {thing}.'

This is a simple string.

#### Knowing Your Fields
This ∆F string consists of six fields: 

1. a __Code__ Field `{what}`, which returns the value of the variable `what`; 
2. a __Text__ Field `" is a "`;
3. another __Code__ Field `{type}`, returning the value of the variable `type`;
4. a short __Text__ Field `" "`; 
5. a __Code__ Field `{thing}`, referencing `thing`; and finally,
6. a final __Text__ Field `"."`.

#### Debug Mode: `'debug' ∆F ...`
We can show each of the fields more graphically using the _debug_ option (abbreviated _d_), which places each field in a separate display box and marks each space in each field by a middle dot `·`.

In [102]:
'd' ∆F '{what} is a {type} {thing}.'

┌────┐┌──────┐┌──────┐┌─┐┌──────┐┌─┐
│This││·is·a·││simple││·││string││.│
└────┘└──────┘└──────┘└─┘└──────┘└─┘

#### Code Fields Are DFNS: `∆F '{⍪⍳3} {↑"Name" "Addr" "Phone"}'`
As shown above, in addition to __Text__ fields, we can create executable __Code__ fields, using braces {...}. A __Code__ field with bare variable names is the simplest type of __Code__ field.

__Code__ fields can be generalized as dfns evaluated in the active (caller's) namespace. While each __Code__ field is executed via ordinary APL rules (statements left-to-right and right-to-left within statements), __Code__ fields within a **∆F** format string are themselves executed left-to-right: 

- the left-most Code field is executed first, then the one to its right, and so on.
     
Each **Code** field has implicit arguments `⍵` and `⍺`:

- ⍵ consists of all the arguments to the full **∆F**. These can be accessed individually 
via ⍹0, ⍹1, etc (see below).
- ⍺ is a reference to a special namespace for sharing
temporary variables across **Code** fields (see below).
     
Each __Code__ field _must_ return a value (perhaps a null string). 

#### Code Field Comments: `{... ⍝...}`

A Code field may include one or more comments. Each comment starts with a lamp `⍝` and contain no braces or ⋄; it terminates just before a following diamond `⋄` or a closing brace `}`. 

For example:

In [103]:
∆F '{ ?0  ⍝ A random number ⍵: 0<⍵<1 }'

0.156232

#### Code Fields Are DFNS: More Complex Examples
Let's look at more complex examples. First, what if a variable itself is more than a simple one-line text string? Unlike format strings in other languages, __`∆F`__ __strings__ handle multi-dimensional objects the _APL way_!

In [104]:
nums←  ⍪⍳3
what←  ↑'This' 'That' 'The other thing'  
type←  ↑'simple' 'hard' 'confusing'
thing← ↑'string' 'matrix' 'thingamabob'
∆F '{nums} {what} is a {type} {thing}'

0 This is a simple string 
1 That hard matrix 
2 The other thing confusing thingamabob

Here, `num` is a column vector of integers, and `what`, `type` and `thing` are character matrices. Any object that can be formatted via Dyalog `⎕FMT` can be returned from a __Code__ field.

Now for a more complex example: you can place arbitrary APL code within the braces `{...}` of a __Code__ field. 

In the example below, we'll remove the ↑ prefix from the values of each of the three variables above. Notice how we insert a period after each word of the variable `thing` and create a quoted string using double quotes: `{ ↑thing,¨"." }` Such a string is called a __DQ String__ and appears only within __Code__ fields. (We'll say more about __DQ Strings__ in a moment.)

In [105]:
what←'This' 'That' 'The other thing'
type←'simple' 'hard' 'confusing'
thing←'string' 'matrix' 'thingamabob'
∆F '{ ⍪⍳≢what }  { ↑what } is a { ↑type } { ↑thing,¨"." }'     

0 This is a simple string. 
1 That hard matrix. 
2 The other thing confusing thingamabob.

#### Code Fields Support Arbitrary Calculations

As you have already seen, `∆F` **Code** fields can be used for arbitrary calculations:

In [106]:
r←1 2 5    ⍝ radius  
script←'If the radius is { ⍪r }m, the circumference is { ⍪○2×r }m, and the area is { ⍪○r*2 }m².'
∆F script 

If the radius is 1m, the circumference is 6.28319m, and the area is 3.14159m².
 2 12.5664 12.5664 
 5 31.4159 78.5398

As indicated above, **Code** fields are executed sequentially from left to right as independent dfns:

In [107]:
ctr←1
∆F 'ctr is {ctr}, now {ctr⊣ctr+←1 }, now {ctr⊣ctr+←1 }, now {ctr⊣ctr+←1 }'

ctr is 1, now 2, now 3, now 4

### DQ Strings Required in Code Fields

#### DQ Strings:  Use `"Double Quotes"` _not_ `'Single Quotes'`
Within __Code__ fields, strings *require* double quotes (`"`). These __DQ strings__ `"like this one"` are used wherever single-quoted strings `'like this'` would be used in standard APL; single-quoted strings are ***not*** used in `∆F` __strings__. Single quotes *may* appear, most usually as literal characters, rather than to create strings.

#### DQ Strings: Escapes, including  `\⋄`... 
__DQ Strings__ support the escaped sequences `\⋄` and `\\`. `\⋄` is a convenient way to enter newlines (actually ⎕UCS 13, the carriage return character) into linear strings (APL character vectors). When the `∆F` _string_ is printed, newlines will create separate _rows_ in the output matrix. <br>(Note: Unlike __Text__ fields, __DQ Strings__ in __Code__ fields do not require or allow braces to be escaped: braces are ordinary characters inside __DQ Strings__.)

To include an actual double quote within a __DQ String__, double the doublequote (`""`), just as one would for _single_ quotes in standard APL strings. (As always, APL of course requires single quotes within character strings to be doubled on entry). Notice how the string below is a 3-row matrix, one row for each line of the __DQ String__.

In [108]:
⍝                     Row 1  Row 2                 Row 3...
  '# rows:', ≢⎕← ∆F '{"This \⋄is a ""DQ"" example,\⋄isn''t it?"}'

This 
is a "DQ" example,
isn't it? 
# rows: 3

### Space Fields
#### `{  }`
The third and last field type is a **Space** field, which looks just like an *empty* <b>Code</b> Field, containing only zero or more spaces between the braces `{` `}` and, optionally, a comment. A **Space** field forms a distinct field and is a good way to separate other sorts of fields, i.e. __Text__ or __Code__ fields.
#### Space Field Comments: `{  ⍝...}`
A **Space** field may include a comment, which starts with a lamp `⍝` and contains ***no*** braces; the comment ends just before the right brace `}` that terminates the **Space** field.
 
 For example:

In [109]:
⍝ Show Space Field comments...                           
    ∆F '{⍪1 2 3}{     ⍝ 5 spaces }{⍪○1 2 3}'     
'd' ∆F '{⍪1 2 3}{     ⍝ 5 spaces }{⍪○1 2 3}' 

1 3.14159
2 6.28319
3 9.42478

┌─┐┌─────┐┌───────┐
│1││·····││3.14159│
│2│└─────┘│6.28319│
│3│ │9.42478│
└─┘ └───────┘

#### Space Field Examples

In [110]:
⍝ ...
    ∆F 'This is{ }a test.'
'd' ∆F 'This is{ }a test.'

This is a test.

┌───────┐┌─┐┌───────┐
│This·is││·││a·test.│
└───────┘└─┘└───────┘

#### Why Space Fields?
But why bother with space fields at all? 
- They are useful when separating one multiline strings or code field from the next; even a zero-width space field can separate two __Text__ fields;  
- They ensure the expected amount of spacing when preceded or followed by text fields with lines of varying length.

Here's an example of two multiline __Text__ field separated by a __Space__ field with a single space: { }.

In [111]:
⍝ ...
    ∆F 'This\⋄is a\⋄multiline\⋄field!{ ⍝ 1 Space}{"This\⋄is\⋄as well!"}'
'd' ∆F 'This\⋄is a\⋄multiline\⋄field!{ ⍝ 1 Space}{"This\⋄is\⋄as well!"}'

This This 
is a is 
multiline as well!
field!

┌─────────┐┌─┐┌────────┐
│This·····││·││This····│
│is·a·····│└─┘│is······│
│multiline│ │as·well!│
│field!···│ └────────┘
└─────────┘

In this next example, we use a zero-width __Space__ field simply to allow us to create two independent __Text__ fields:

In [112]:
∆F '1. \⋄2.\⋄3.{}Jane\⋄John\⋄Nancy'   

1. Jane 
2. John 
3. Nancy

This is equivalent, with an explicit Space field of length 1.
```   
 ∆F '1.\⋄2.\⋄3.{ }Jane\⋄John\⋄Nancy'
```
#### Space Fields with explicit width counts:  `{  :30: }`
Sometimes it's a bit inconvenient to count out a field width or to specify a wide **Space Field**. If so, rather than typing the actual spaces, simply enter inside the braces the desired number of spaces ***as an integer*** between two colons (possibly followed by a comment). The colons are  there to avoid confusion with **dfn** syntax. Each explicit width count must between ***0 and 999***.

Here we separate each **Code Field** by exactly five spaces, encoded *slightly* differently:

In [113]:
⍝ ...         12345        Five         Still Five Spaces
    ∆F '{⍪1 2}     {⍪○1 2}{ :5: }{⍪*1 2}{:5:⍝Five spaces}{⍪(○1 2)×*1 2}'  
'd' ∆F '{⍪1 2}     {⍪○1 2}{ :5: }{⍪*1 2}{:5:⍝Five spaces}{⍪(○1 2)×*1 2}'

1 3.14159 2.71828 8.53973
2 6.28319 7.38906 46.4268

┌─┐┌─────┐┌───────┐┌─────┐┌───────┐┌─────┐┌────────┐
│1││·····││3.14159││·····││2.71828││·····││·8.53973│
│2│└─────┘│6.28319│└─────┘│7.38906│└─────┘│46.4268·│
└─┘ └───────┘ └───────┘ └────────┘

### Code Fields: Advanced Topics
### Pseudobuiltin `$` for Detailed Formatting
Dyalog APL has a built-in formatting utility `⎕FMT`, which may be old-fashioned but is rather essential for precise formatting of numerical data (and sometimes text data). To make `⎕FMT` and its standard parameters easier to use, we provide it as the pseudo-builtin `$`, with extensions described below. Here's a typical example, where the (optional) formatted left argument (⍺) is placed within a DQ string (as required with `∆F` **Code** Fields):

In [114]:
∆F '{ "F8.6" $ ?3⍴0  ⍝ Random Floats }  {"ZI2,⊂/⊃,ZI2,⊂/⊃,ZI4" $ ⍉⍪⎕TS[2 1 0] ⍝ a European Style Date}'

0.121620 25/12/2021
0.943754 
0.851188

Keep in mind that `$`, like `⎕FMT`, treats simple vectors as column vectors (1-column matrices) ***by default***. (**∆F** has a way to override this default: see below.)

Let's move on to another example with `⎕FMT` features you may have forgotten.

In [115]:
∆F 'Multiples of pi: {"I1,⊂×π =⊃" $ 1+⍳4} {"F10.7,⊂…⊃" $ ○1 2 3 4}'

Multiples of pi: 1×π = 3.1415927…
 2×π = 6.2831853…
 3×π = 9.4247780…
 4×π = 12.5663706…

Again, using the *debug* option, we can see exactly what fields are set up.

In [116]:
'd' ∆F 'Multiples of pi: {"I1,⊂×π =⊃" $ 1+⍳4} {"F10.7" $ ○1 2 3 4}'

┌─────────────────┐┌─────┐┌─┐┌──────────┐
│Multiples·of·pi:·││1×π·=││·││·3.1415927│
└─────────────────┘│2×π·=│└─┘│·6.2831853│
 │3×π·=│ │·9.4247780│
 │4×π·=│ │12.5663706│
 └─────┘ └──────────┘

### Pseudobuiltin \$$ for a Boxed Display
If we want a __Code__ field to be boxed in the regular output, we can use the pseudo-builtin function __\$\$__. By default (no left argument or a left-argument of 0), no middle dots (·) appear with \$\$. If you want middle dots to appear in place of spaces, you must provide a left argument of 1.

In [117]:
OVER←  (⍪,⍥⊂)                                        ⍝ A Helper Function
boats← (↑'Nina' 'Pinta' 'Santa Maria')  

title← ∆F 'Default (Spaces) { :7: } With Middle Dots'
bfmt←  ∆F ' { $$ ⍵1} {:10:} { 1 $$ ⍵1 }' boats       ⍝ Hold on! We explain ⍵1 just below.
title OVER bfmt

Default (Spaces) With Middle Dots 
 ┌───────────┐ ┌───────────┐ 
 │Nina │ │Nina·······│ 
 │Pinta │ │Pinta······│ 
 │Santa Maria│ │Santa·Maria│ 
 └───────────┘ └───────────┘

### Shortcuts for Accessing Elements of ⍵
#### ∆F Names `⍹0` ... `⍹99` and `⍹`   (or `⍵0` ... `⍵99` and `⍵_`)
__∆F__ allows __Code__ fields to access elements in its right argument, including the format string itself. Elements here refer to **top level scalar objects** in the right argument `⍵` to __∆F__, normalized to a nested form `⊆⍵`, ordered scalar by scalar (in ⎕IO=0) as (0⊃⍵) (1⊃⍵) (2⊃⍵) ... (N⊃⍵):

- The format string itself `(0⊃⍵)` is abbreviated as `⍹0`, `(1⊃⍵)` as `⍹1`, ..., (N⊃⍵) as `⍹N`.
- If a Code field refers to an element that does not exist, a runtime `INDEX ERROR` signal is generated as expected:
```
      ∆F '{⍹2}' 1 
   ∆F INDEX ERROR: ⍹2 is out of range.
```
- The character `⍹` (omega underscore) is `⎕UCS 9081`. 
- If `⍹` is not handy, there are alternatives using a simple omega `⍵`: 
  - for omega underscore with numeric suffixes, use: 
    - `⍵0` for `⍹0`, `⍵10` for `⍹10`, etc.
  - for bare omega underscore (next element-- see below), use: 
    - `⍵_` for `⍹`. That's a ⍵ followed by an underscore _.
    
Here is an example accessing `⍹1`, shorthand for `(1⊃⍵)`.

In [118]:
∆F '{⍹1} multiples of pi: {"I1,⊂×π =⊃" $ 1+⍳⍹1 } {"F10.7" $ ○ 1+⍳⍹1}'  3  
∆F '{⍹1} multiples of pi: {"I1,⊂×π =⊃" $ 1+⍳⍹1 } {"F10.7" $ ○ 1+⍳⍹1}'  2  

3 multiples of pi: 1×π = 3.1415927
 2×π = 6.2831853
 3×π = 9.4247780

2 multiples of pi: 1×π = 3.1415927
 2×π = 6.2831853

The symbol `⍹` used alone in a __Code__ field will select the _next_ argument in sequence:

- ⍹ means ⍹1 if this is the first use in any Code field of either an explicit ⍹N or bare ⍹, else 
- ⍹ means ⍹N+1 if ⍹N was the most recently accessed: 
    - For example, ⍹ references ⍹5 if ⍹4 was the most recently accessed, explicitly or implicitly. 
        
This makes it easy to format a set of items: 

In [119]:
w1F← 'F4.2,⊂%⊃'
w2D← 2.200 3.834 5.996
w3F← '⊂£⊃,F7.2'
w4D← 1000.23, 2250.19 2500.868
⍝          ⍵1  ⍵2               ⍵3  ⍵4   ⍵1   ⍵2   ⍵3    ⍵4
∆F 'Rate: {⍹ $ ⍹}; cur. value: {⍹ $ ⍹}'  w1F  w2D  w3F   w4D 

Rate: 2.20%; cur. value: £1000.23
 3.83% £2250.19
 6.00% £2500.87

Note also that `⍹0` (the format string itself) can never be selected via the lone `⍹` (as the _next_ ⍵ argument), since the *last* index specified is never less than 0, so the *next* is never less than 1. In short, if you want `⍹0`, you type it explicitly!

### Justification and Centering with \$

#### Pseudo-builtin `$`: Justification and Centering Codes  _L_, _C_, _R_ <br>Vector Arg treated as Column Vector  
The pseudo-function `$` has been extended with special codes for justifying or centering objects within a __Code__ field. (Codes _l_, _c_, _r_— are discussed further below.)

<table align="left">
    <caption><center>Codes <i>L, C, R</i></center></caption>
<thead>
    <tr><th>Justif.<br>Type</th><th>Col Vec<br>Preferred</th><th>Row Vec<br>Preferred</th></tr>
</thead>
<tbody>
    <tr><td>left</td><td><b>L</b><i>nnn</i></td><td><b>l</b><i>nnn</i></td></tr>
    <tr><td>center</td><td><b>C</b><i>nnn</i></td><td><b>c</b><i>nnn</i></td></tr>
    <tr><td>right</td><td><b>R</b><i>nnn</i></td><td><b>r</b><i>nnn</i></td></tr>
</tbody>
</table>

The digits `nnn`, a 1- to 3-digit number, indicates the minimum width of the field. If a field is already wider than `nnn` characters, the field is left as is.
These codes are valid with either *numeric* or *text* arguments.  Only one special code may be used in each `$` call, but you may call `$` itself more than once) and that code must be the *first* or *only* code specified. If other (usually numerically-oriented) codes  follow, a comma must intervene (following the conventions of dyadic `⎕FMT`).

Here, we *left-, center-*, and *right-* justify the variable `names` using ∆F justification/centering codes.

In [120]:
names←↑'John' 'Mary'
     ∆F '<{"L10" $ ⍵1}> <{"C10" $ ⍵1}> <{"R10" $ ⍵1}>' names  
'd'  ∆F '<{"L10" $ ⍵1}> <{"C10" $ ⍵1}> <{"R10" $ ⍵1}>' names  

<John > < John > < John>
 Mary Mary Mary

┌─┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌─┐
│<││John······││>·<││···John···││>·<││······John││>│
└─┘│Mary······│└───┘│···Mary···│└───┘│······Mary│└─┘
 └──────────┘ └──────────┘ └──────────┘

Here, we format a couple of numeric fields, one centered automatically via $ code `C25` and the other manually via a _standard_ __⎕FMT__ code `X6`, which adds explicit spacing to build the same field width. Both do the job:

In [121]:
title← ∆F '{    }Use ∆F Extension C25\⋄   25 chars wide{ :13: }Use ⎕FMT X6: 6+13+6=25\⋄    25 chars wide'
centr← ∆F '{1$$ "C25,ZF13.9" $ ⍵1 } versus {1$$ "X6,ZF13.9,X6" $ ⍵1 }' (○2 20 300)
title OVER centr

Use ∆F Extension C25 Use ⎕FMT X6: 6+13+6=25 
 25 chars wide 25 chars wide 
 ┌─────────────────────────┐ versus ┌─────────────────────────┐ 
 │······006.283185307······│ │······006.283185307······│ 
 │······062.831853072······│ │······062.831853072······│ 
 │······942.477796077······│ │······942.477796077······│ 
 └─────────────────────────┘ └─────────────────────────┘

Note also the use of the explicit digits in the **Space** field `{  :13:  }` to insert a significant set of spaces to separate the titles.

#### Pseudo-builtin `$`: Justification and Centering Codes _l_, _c_, _r_  <br>Vector arg treated as _Row_ Vector
Like standard ⎕FMT, `$` _by default_ considers simple vectors in the code field as column vectors (as in the example above). This is true even for the extensions `L`, `C`, and `R`. However, you can override this, by specifying justification codes in lower case (`nnn` is a 1- to 3-digit number):

<table align="left">
     <caption><center>Codes <i>l, c, r</i></center></caption>
<thead>
    <tr><th>Justif.<br>Type</th><th>Col Vec<br>Preferred</th><th>Row Vec<br>Preferred</th></tr>
</thead>
<tbody>
    <tr><td>left</td><td><b>L</b><i>nnn</i></td><td><b>l</b><i>nnn</i></td></tr>
    <tr><td>center</td><td><b>C</b><i>nnn</i></td><td><b>c</b><i>nnn</i></td></tr>
    <tr><td>right</td><td><b>R</b><i>nnn</i></td><td><b>r</b><i>nnn</i></td></tr>
</tbody>
</table>

If these are used, simple vectors in the code field used as arguments to `$` are treated as 1-row matrices instead. 
- Right arguments that are not simple or are not a numeric or character vector are not impacted.

Here, `"c0"` (or `"l0"` or `"r0"`) formatting with `$` ensures a simple vector right argument (numeric or character) is treated as a 1-row matrix. Similarly, `"C0"` (or `"L0"` or `"R0"`) formatting with `$` ensures a simple vector right argument (numeric or character) is treated as a column vector, even if no standard ⎕FMT codes are used. 
- The codes `"c0"`, `"C0"`, _et cetera_ do the job because justification and centering codes ensure a __minimum__ width, never truncating fields over that width. 

In [122]:
∆F 'For n∊1 2 3, nπ = { "c0" $ ○ ⍵1 }.   {"I1,⊂π = ⊃" $ ⍵1} { "C0" $ ○⍵1 }' (1 2 3)

For n∊1 2 3, nπ = 3.14159 6.28319 9.42478. 1π = 3.14159
 2π = 6.28319
 3π = 9.42478

### Self-Documenting Code Fields 

#### `{ Code → }`

As mentioned earlier, a **Code** Field can be made **self-documenting** by inserting a right arrow `→` just before the closing right brace.  In more detail, the entire code of the **Code** Field, including the right arrow *and* the spaces _before_ and _after_ it, will be included in the formatted output, followed by the executed code. For clarity,, the APL right arrow `→` is replaced by a special right arrow `➤ `. 

In [123]:
MyString← ↑'This' 'is my' 'string'
Today←  ⍪⎕TS 
∆F '<{MyString → }>  <{ 3↑Today → }>.  <{"I4" $ 3↑Today  →  }>'

<MyString ➤ This > < 3↑Today ➤ 2021>. <"I4" $ 3↑Today ➤ 2021>
 is my 12 12 
 string 25 25

<font color='red'>**Warning:**</font> **Jupyter Notebook** seems to improperly format some APL output, including output from self-documenting **Code** fields. This does not occur in a Dyalog APL **Ride** session.

##### Comments in Self-Documenting Code Fields

Comments are allowed in **Self-documenting Code Fields**, but must be terminated by a diamond `⋄`; the final right arrow appears just before the right brace that closes the **Code** Field. For example:  

In [124]:
∆F ' { ⎕AV ⍳"aeiou" ⍝ Find the "vowels" in ⎕AV ⋄  → }  '

⎕AV ⍳"aeiou" ⍝ Find the "vowels" in ⎕AV ⋄ ➤ 17 21 25 31 37

### Pseudobuiltin `$$$` for Adding Quotes or Decorators


In [125]:
guillemets← '«»'             ⍝ guillemets: French-style quotes!
gUnicode←   171 187          ⍝ Unicode for «»
∆F '{$$$ ⍵1 ⍝ Default qts}     {guillemets $$$ ⍵1}     {gUnicode $$$ $ ⍵1}' (⍪100×⍳3)

"0" «0» «0»
"100" «100» «100»
"200" «200» «200»

In [126]:
animals←↑'cats' '  rats'  'dogs  '        ⍝ Note arbitrary blanks within each row of animals
∆F '{animals  } {:10:} { $$$ animals  } {:10:} { """",animals,""""}'

cats "cats" "cats "
 rats "rats" " rats"
dogs "dogs" "dogs "

Here's a clever use of `$$$`.

Unlike `"I2"` (a **⎕FMT** specification), `$$$` automatically adjusts to the width of the numbers being formatted. (**⎕FMT** will insert asterisks (***), if the field width is insufficient.

In [127]:
∆F '{ " ." $$$ ⍪⍵1}  { "I2,⊂.⊃" $ ⍵1}  { "I3,⊂.⊃" $ ⍵1}  ' (98 99 100)

98. 98. 98. 
 99. 99. 99. 
 100. **. 100.

### Pseudobuiltin % for Formatting Dates and Times

As Dyalog defines it, a *datetime* is a date and time of day represented by a timestamp, a date number, or a "military" timezone character. Dyalog APL provides two very expansive datetime functions, `⎕DT` and `(1200⌶)`, to manipulate datetimes. The `∆F` pseudofunction `%` uses these functions _behind the scenes_ to provide a simple way to format one or more APL date-time specifications:

- APL **Timestamps** 
   - e.g. `⊂⎕TS` which might produce **2021 06 02 14 52 05 443**, 

- Dyalog **Date Numbers** 
   - e.g. `1 ⎕DT ⊂⎕TS` which (at the same time as above) would produce the Dyalog Date Number **44348.61951**,

- **Timezone Characters**
   - e.g. `"J"`, which refers to the local timezone, or `"Z"` which refers to Coordinated Universal Time (UTC or Greenwich Mean Time).

For the specific datetime character codes for the left argument of `%`, see`(1200⌶)`.

Here are a few simple examples of how `%` is used.

#### Using scalar Timestamps and Date Numbers

In [136]:
⍝ t1 is a Timestamp in APL format
  t1←2021 06 02 14 52 05 443
⍝ d1 is the Date Number equivalent to timestamp t1
  'd1: ',(d1← 1 ⎕DT ⊂t1)
⍝ tf: Our datetime format string...
  tf← 't:mm PP "on the" _Doo "of" Mmm YYYY"."'
⍝ t2 is a Timestamp from history...
  t2←1776 07 04 18 05
  
⍝ These are equivalent, since t1 and d1 refer to the same date and time.
  ∆F'Let''s meet at {  tf % ⍵1 }' (⊂t1)
  ∆F'Let''s meet at {  tf % ⍵1 }' d1
⍝ Now, we'll format two timestamps into two rows...
  ∆F'We\⋄They{ }met at\⋄met at{ }{ ↑ tf  % t1 t2 }' 

d1: 44348.6

Let's meet at 2:52 PM on the 2nd of Jun 2021.

Let's meet at 2:52 PM on the 2nd of Jun 2021.

We met at 2:52 PM on the 2nd of Jun 2021.
They met at 6:05 PM on the 4th of Jul 1776.

#### Using a Timezone Character
And here's an example using the ISO (International Standards Organization) format `%ISO%` on the left and "Zulu" (UTC) Time on the right::

In [129]:
⍝ Here we specify the current datetime in timezone Zulu, i.e. Greenwich Mean Time or UTC.
  ∆F'Product A Expired on { "%ISO%" % "Z" } UTC.'

Product A Expired on 2021-12-25T20:05:19 UTC.

#### Performing Datetime Manipulation

In [130]:
  ndays← ¯14
⍝ Add "ndays" to the current Zulu time...
  ∆F'Product B Expired {|ndays} Days Earlier, on: { "%ISO%" %  ndays+1 ⎕DT "Z" } UTC.' ndays

Product B Expired 14 Days Earlier, on: 2021-12-11T20:05:20 UTC.

### Code Fields: Private Namespace ⍺
**∆F** generates a special namespace, which is passes to each **Code** field as the left argument `⍺` . It allows the user to have functions or variables (e.g. state variables) that are available to all **Code** fields (left to right) during the execution of **∆F**, without cluttering the calling environment:

In [131]:
∆F '{⍬⊣⍺.RND←?3⍴⍵1}The winning numbers are {⍺.RND[0]}, {⍺.RND[1]}, and {⍺.RND[2]}.' 1000

The winning numbers are 377, 802, and 768.

### ∆F and Looping

While languages like Python might tend to examples requiring explicit loops like this:

``# Python ...
table = [4127, 4098, 7678]
for num in table2:
    print(f'{num:10}')
 ``
 
 APL can typically format an entire object in a single statement and in a readable format:

In [132]:
table← 4127 4096 7678                           ⍝ Could be much bigger, of course...
∆F '{"[]" $$$ ⍪1+⍳≢⍵1}  {⍪⍵1}' table            ⍝ $$$ defined above...

[1] 4127
[2] 4096
[3] 7678

Or, compare this Python example:

``# Python
print(f'Number Square Cube')
for x in range(1, 11):
     print(f'{x:2d} {x*x:3d} {x*x*x:4d}')
 ``
 
and its APL equivalent below. *(Of course, a simple Dyalog ⎕FMT statement would work perfectly and concisely in this case!)*

In [133]:
rangeV ← 1+⍳10
Head←'Number     Square     Cube'
Head OVER ∆F  '  {"I2" $ ⍵1}        {"I3" $ ⍵1*2}       {"I4" $ ⍵1*3}'  rangeV

Number Square Cube 
 1 1 1 
 2 4 8 
 3 9 27 
 4 16 64 
 5 25 125 
 6 36 216 
 7 49 343 
 8 64 512 
 9 81 729 
 10 100 1000

Keep in mind as you read examples from Python or Javascript that APL is likely to have more intuitive solutions that do not require explicit looping with **∆F**.

## Assertions with ∆F

Normally, ∆F returns the formatted text as a single formatted matrix (rank 2).
<br>If the left argument (⍺) to ∆F is a homogeneous numeric array, it is viewed as an assertion.
  * If the assertion contains _no numeric zeroes_, it is considered __true__. It **prints** the formatted text, returning a shy _1_. (It does _not_ return the formatted text, as in _format_ mode.)
  * If the assertion contains one or more zeroes, it is considered __false__. It does nothing, quickly returning a shy _0_.

In [134]:
⍝ Here, (var<100) is FALSE, so no ∆F string message is produced. 0 is returned.
  var←100
  rc←(var<100) ∆F 'Warning! Variable "var" is out of expected range: var={var}'
  'Assertion formatted (and printed) nothing and returned SHY',rc

Assertion formatted (and printed) nothing and returned SHY 0

In [135]:
⍝ Now, the assertion (var<100) is TRUE, so a ∆F string message is printed. 1 is returned.
⍝ We'll show it explicitly, even though it is shy.
  var←50
  rc←(var<100) ∆F 'Warning! Variable "var" is out of expected range: var={var}'
  '∆F formatted and printed text (via ⎕←) and returned SHY',rc

Warning! Variable "var" is out of expected range: var=50

∆F formatted and printed text (via ⎕←) and returned SHY 1

***
## ∆F Syntax Summary 

Syntax:  **result ← [ [ assertion | options ] ]  ∆F  format_string  [ arg1 [arg2 ...] ]**

- **assertion** ⍺: a homogeneous numeric array. 
     - The assertion is "TRUE", unless it contains at least one 0.
       - If TRUE, **∆F** prints the formatted result and returns a shy 1. 
       - If FALSE, **∆F** does nothing, returning a shy 0.
     
- **options** ⍺: DEBUG | COMPILE | HELP | DEFAULT*         
     - DEBUG: Displays each field separately (via dfns "box")
       The abbrev 'DE' or 'D' denotes DEBUG. 
     - COMPILE: Returns a code string that can be converted to a dfn (executed via ⍎), 
        rather than scanned on each execution. 
        - See **COMPILE option** below.
     - HELP: Displays HELP documentation (⍵ ignored):  `∆F⍨'HELP'`
     - DEFAULT: Returns a formatted 2-D array according to the format_string 
       specified.
              
      * If ⍺ is omitted or ⍺ is a null string `''`, option DEFAULT is assumed.
     - Options may be in either case and abbreviated.  
       
- **format_string**  `⍹0` (0⊃⍵)
     - Contains the simple format "fields" that include strings (**Text** fields), 
       code (**Code** fields), and spacing (**Space** fields). 
       <br>**Code** fields support a shorthand for manipulating objects using **\$**, **\$\$**, **\$\$\$**, and **\%**:
       
       - **$** does numeric formatting via **⎕FMT**, along with justification and centering;
       
       - **$$** displays data using dfns **DISPLAY**;
       
       - **$$$** adds quotes or other decorators around user data.
       
       - **%** formats timestamps and date numbers using **⎕DT** and **1200⌶**.
       
 - arbitrary **args** `1↓⍵`
     - Optional arguments to be referenced in Code Fields. 
     - Shorthand in Code Fields
        - **⍹0** refers to `0⊃⍵`, the ∆F format string.
        - **⍹1** refers to `1⊃⍵`, the first "user" element.
        - **⍹N** refers to `N⊃⍵`, the Nth "user" element (where N is a 1- or 2-digit number)
        - **⍹**  by itself refers to the element after the one most recently accessed across all code fields viewed from left to right. If no element has been accessed, **⍹** refers to **⍹1**.
                
     - Allowed synonym for **⍹**: You may use `⍵0` for **⍹0**, `⍵N` for **⍹N**, and `⍵_` for a **⍹** by itself.
        
- **result**
     - For an assertion, result is either a shy 1 (TRUE) or 0 
       (FALSE, where *at least* one element of ⍺ was 0).
     - The final formatted object is printed (via ⎕←) if the result is TRUE.
     - Otherwise, if `DEFAULT` formatting is specified (*i.e.* the option `COMPILE` is not used), 
       result will be a matrix containing all the fields glued together.
     - If the `COMPILE` option is specified, result will be a code string that 
       can be executed to produce the result either of an assertion (see above) 
       or the `DEFAULT` formatting. It may be executed immediately via **⍎** or converted 
       to a fn (via **⍎**) once and then called, e.g. in an implicit or explicit loop. 
       
           myFormat←  ⍎'COMPILE' ∆F '...Make use of variable myVar ...'
           :FOR myVar :in CreateVars 1E4
              myFormat '' elem1 elem2 ...
           :ENDFOR  
           
- **COMPILE option**: Given a dfn generated via a call to **∆F** via the **COMPILE** option...
    - The first (or only) element of its **right** argument `⍵` may be 
       - a nullstring `''`
         - *i.e.* a *dummy* format string which will be ignored; the *original* format string presented at compile time will be available as `⍹0`
       - an alternate format string specified at execution time 
         - which will be available instead as `⍵0`.
       - In general: 
           myFun← ⍎ 'compile' ∆F 'my code'
           (myFun '' [⍹1 ... ])    ≡    ('default' ∆F 'my code' [⍹1 ... ])  
    - Its **left** argument `⍺` is omitted, unless the formatting dfn is an assertion. 
      - If ⍺ is present, it must be an **assertion** (see above) with simple numeric data.
 
 <div class="alert alert-block alert-info">
Note: <font color="purple"><b>∆F</b></font> is a <font color="red"><b><i>prototype</i></b></font>, depending heavily on regular expressions (via <b>⎕R</b> and <b>⎕S</b>, including some rather recursive patterns) so it's rather <font color="red"><b><i>slow</i></b></font>. <b>Compiled</b> <font color="purple"><b>∆F</b></font> strings (namely, associated <i><b>dfns</b></i> generated via the <i>compile</i> option) are several-fold faster than <b>standard</b> <font color="purple"><b>∆F</b></font> strings; they may be useful in loops or oft-repeated expressions. Nothing substitutes, however, for moving from a <font color="red"><b><i>prototype</i></b></font> version to <font color="red"><b><i>production</i></b></font> code.
</div>

### [Go to top](#∆F-Strings-for-Formatting)